In [ ]:
!pip install transformers

import tensorflow as tf
import numpy as np
import sklearn
from sklearn.metrics import precision_score,balanced_accuracy_score,recall_score,f1_score
import transformers
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import json
#import matplotlib.pyplot as plt
import random
#import seaborn as sn
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

batch_size=32 * tpu_strategy.num_replicas_in_sync
print('Batch size:', batch_size)
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
labels,headlines,texts = [],[],[]

def read_jsondata(filename):
  data = [json.loads(line) for line in open(filename,'r')]
  random.shuffle(data)
  for line in data:
    labels.append(line['category'])
    headlines.append(line['headline'])
    text=line['headline']+' '+line['short_description'] 
    texts.append(text)
  return data
##driver function
filename = '/content/drive/MyDrive/News_Category_Dataset_v2.json'
data = read_jsondata(filename)
n_elements=len(headlines)

def aggregate_categories(labels):
    aggregated=[]
    for line in labels:
        if line=='WORLDPOST' or line=='THE WORLDPOST':
            aggregated.append('WORLD NEWS')
        elif line=='PARENTING' or line=='PARENTS':
            aggregated.append('PARENTS & PARENTING')
        elif line=='ARTS' or line=='CULTURE & ARTS':
            aggregated.append('ARTS & CULTURE')
        elif line=='STYLE':
            aggregated.append('STYLE & BEAUTY')
        elif line=='GREEN' or line=='ENVIRONMENT':
            aggregated.append('ENVIRONMENT & GREEN')
        elif line=='HEALTHY LIVING' or line=='WELLNESS' or line=='HOME & LIVING':
            aggregated.append('WELLNESS & HEALTHY LIVING')
        elif line=='COMEDY' or line=='ENTERTAINMENT' or line=='MEDIA' or line=='TRAVEL' or line=='WEDDINGS':
            aggregated.append('ENTERTAINMENT & FUN')
        elif line=='TASTE' or line=='FOOD & DRINK':
            aggregated.append('FOOD, DRINK & TASTE')
        elif line=='COLLEGE' or line=='EDUCATION':
             aggregated.append('COLLEGE & EDUCATION')
        elif line=='SCIENCE' or line=='TECH':
             aggregated.append('SCIENCE & TECH')
        elif line=='BUSINESS' or line=='MONEY':
            aggregated.append('BUSINESS & MONEY')
        elif line=='BLACK VOICES' or line=='GOOD NEWS' or line=='LATINO VOICES' or line=='QUEER VOICES':
            aggregated.append('IMPACT')

        else:
            aggregated.append(line)
    return aggregated


##
aggregated=aggregate_categories(labels)
categories=sorted(list(set(aggregated)))
n_categories=len(categories)

def indicize_labels(labels):
    """Transforms string labels into indices"""
    indices=[]
    for j in range(len(labels)):
        for i in range(n_categories):
            if labels[j]==categories[i]:
                indices.append(i)
    return indices

def tokenize(modelname):
  tokenizer = AutoTokenizer.from_pretrained(modelname)
  
  inputs = tokenizer(texts,padding = True,truncation=True,return_tensors='tf')
  dataset=tf.data.Dataset.from_tensor_slices((dict(inputs), indices))
  return dataset

##
indices=indicize_labels(aggregated) 
dataset = tokenize("bert-base-cased") 

def train_test_ds():
  val_data_size=int(0.1*n_elements)
  val_ds=dataset.take(val_data_size).batch(batch_size, drop_remainder=True) 
  train_ds=dataset.skip(val_data_size).batch(batch_size, drop_remainder=True)
  train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
  return train_ds,val_ds,val_data_size
##
train_ds,val_ds,val_data_size = train_test_ds()




Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

In [7]:
def train(modelname,train,val):
  with tpu_strategy.scope():
    model = TFAutoModelForSequenceClassification.from_pretrained(modelname, num_labels=n_categories)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5, clipnorm=1.),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.metrics.SparseCategoricalAccuracy(),tf.keras.metrics.SparseTopKCategoricalAccuracy(k=3, name='Sparse_Top_3_Categorical_Accuracy')]
        )
    
  return model

##
model = train('bert-base-cased',train_ds,val_ds)


Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, min_delta=0.0001)
#mc = ModelCheckpoint('./best_model.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
model.fit(train_ds, validation_data=val_ds, epochs=1,verbose=1)

In [ ]:
#from keras.models import load_model
#model1 = model.load_weights('./saved_weights.h5')
#model=load_model('best_model.h5')

In [9]:
model.save_pretrained("/content/drive/MyDrive/saved_pretrained")

In [8]:
model1 = model.from_pretrained("/content/drive/MyDrive/saved_pretrained")

Some layers from the model checkpoint at /content/drive/MyDrive/saved_pretrained were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at /content/drive/MyDrive/saved_pretrained.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [ ]:
model.save('./MyModel_h5.h5', save_format='h5')
# loading the saved model
loaded_model_h5 = tf.keras.models.load_model('./MyModel_h5.h5')